## Experiment with query semantic vector for caching

<small>
(from <a href="http://maven.com/softwaredoug/cheat-at-search">Cheat at Search with LLMs</a> training course by Doug Turnbull.)
</small>

Our cache:

# Vector DB:

```
"red tennis shoes", ->
  {
    "color": "red",
    "itemType": "tennis shoe",
    "category": "footwear"
  }
```

# Query

```
crimson shoes for tennis
```

1. Find an embedding model that captures our domain
2. Decide on a threshold

### A set of highly related queries

These queries vary a couple of topics, but all are different strings for the same information needs

* Color: red or purple
* Item type: shoes or tennis shoes
* Gender: men or womens

Below we have two sets of test queries:
* Queries that should NOT match - they have different, if similar, information needs
* Queries that its OK if they match

We will use an embedding model fine-tuned on e-commerce data: `intfloat/e5-small-v2`. You will need to find an embedding model that passes this test.

In [ ]:
from sentence_transformers import SentenceTransformer

query_eggshells = [
    "red tennis shoes",
    "tennis shoes, the red ones",
    "I want to buy red tennis shoes",
    "please give me red tennis shoes",
    "red shoes",
    "tennis shoes",
    "red shoes for men",
    "red shoes for women",
    "I want to buy red shoes",
    "I want to buy red shoes for men",
    "purple shoes",
    "I want to buy purple shoes",
    "purple tennis shoes",
    "I need purple tennis shoes",
    "tennis shoes in purple",
    "can I get some purple tennis shoes?",
    "I'd like to get some red shoes",
    "red women's tennis shoes",
    "men's red tennis shoes",
    "red shoes men's",
    "shoes, red, for women",
    "give me tennis shoes in red",
    "looking for red shoes for women",
    "shopping for red men's tennis shoes",
    "purple women's shoes",
    "purple men's tennis shoes",
    "I’m shopping for tennis shoes, maybe in red",
    "tennis shoes that are red",
    "show me red tennis shoes for women",
    "do you have red tennis shoes?",
    "red colored shoes",
    "red sneakers",
    "purple sneakers for men",
    "men's purple shoes",
    "women's red sneakers",
    "I'd like women's tennis shoes in red",
    "get me red tennis shoes",
    "can I buy red shoes?",
    "I need tennis shoes for men in red",
    "buy women's tennis shoes purple",
    "want to see red men's shoes",
    "looking for purple tennis shoes for women",
    "show me tennis shoes, red color",
    "are there purple shoes for women?",
    "tennis shoes for women in purple",
    "ladies red tennis shoes",
    "guys' red shoes",
    "purple tennis shoes for ladies",
    "purple shoes for guys",
    "red tennis sneakers",
    "tennis shoes — red ones for women",
    "searching for red sneakers",
    "tennis shoes red male",
    "red tennis shoes female",
    "I’m looking for red sports shoes",
    "purple tennis shoes men's",
    "tennis sneakers in purple for women",
    "buy red sneakers",
    "purple athletic shoes for men",
    "purple gym shoes",
    "red gym shoes",
    "ladies tennis shoes, red color",
    "purple tennis footwear for men",
    "men's gym shoes in red",
    "I'd like to see red shoes for women",
    "I need new red tennis shoes",
    "let’s buy red shoes",
    "do you carry red tennis sneakers?",
    "want red running shoes",
    "get purple tennis sneakers",
    "red tennis shoes for women",
    "red shoes in men’s sizes",
    "purple colored shoes for women",
    "do you have men’s tennis shoes in red?",
    "purple shoes for women please",
    "can I find red women’s sneakers?",
    "I want red tennis shoes for men",
    "men’s shoes in red",
    "show red tennis shoes",
    "women's red running shoes",
    "purple tennis shoes women want",
    "buy men's purple shoes",
    "buy women’s red sneakers",
    "tennis shoes with red color",
    "tennis shoes for ladies in red",
    "give me red athletic shoes",
    "I’m after red tennis sneakers",
    "purple workout shoes",
    "purple tennis shoes I can run in",
    "can I buy red women's shoes?",
    "buy shoes in red for women",
    "looking for tennis shoes, red ones",
    "I like purple tennis shoes",
    "tennis sneakers in red for men",
    "ladies' red shoes",
    "running shoes, red, for women",
    "men’s purple sports shoes",
    "tennis shoes, purple, male",
    "tennis sneakers purple female",
    "purple shoes for sports",
    "purple shoes for tennis",
    "get me red shoes for gym",
    "I’d like tennis shoes, in red"
]

e_commerce = 'intfloat/e5-small-v2'
# minilm = 'sentence-transformers/all-MiniLM-L6-v2'
model = SentenceTransformer(e_commerce)
prefix = "query: "
query_embeddings = model.encode([prefix + query for query in query_eggshells])
print(query_embeddings)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: a3fb316a-268e-4d78-b953-80f4b7298416)')' thrown while requesting HEAD https://huggingface.co/intfloat/e5-small-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


[[-0.07041039  0.05990096  0.00384035 ...  0.00889128 -0.01869312
   0.07684675]
 [-0.06730099  0.06927424  0.01202622 ...  0.01739461 -0.0057798
   0.07632535]
 [-0.07761967  0.04985245  0.0105322  ... -0.0104816  -0.03585915
   0.06742807]
 ...
 [-0.10389227  0.03124737  0.01955353 ... -0.01935114  0.00356002
   0.07294314]
 [-0.09446127  0.03613773  0.01525081 ...  0.01329067 -0.07370114
   0.04785777]
 [-0.06866553  0.05815653 -0.00062291 ... -0.00144215 -0.02783558
   0.06176304]]


### Our 'caching' function

This will return a hit if similarity with our embedding model exceeds a threshold, otherwise it will not be considered a match.

In [ ]:
import numpy as np

def similar_query(other_query_embeddings, query, threshold=0.95):
    query_embedding = model.encode(prefix + query)
    scores = np.dot(query_embedding, other_query_embeddings.T)
    max_idx = np.argmax(scores)
    if scores[max_idx] < threshold:
        return None, None
    return query_eggshells[max_idx], scores[max_idx]

### These should not match

In [ ]:
should_not_match = [
    "show me brown shoes",
    "I'm looking for brown shoes for men",
    "brown shoes for women please",
    "I'd like some brown tennis sneakers",
    "where can I buy brown tennis shoes for women?",
    "get me some tennis shoes, brown ones",
    "tennis sneakers in brown for men",
    "can I find brown sports shoes for women?",
    "looking for brown men's tennis shoes",
    "do you carry women's tennis shoes in brown?",
    "shopping for brown tennis sneakers",
    "brown sneakers for women",
    "brown gym shoes for men",
    "I'd like tennis shoes in brown",
    "please show me brown shoes",
    "tennis shoes, brown color for ladies",
    "can I get brown shoes?",
    "brown colored sneakers",
    "men’s shoes in brown",
    "ladies’ brown tennis sneakers",
    "tennis shoes in brown for women",
    "give me brown sneakers",
    "brown tennis footwear for men",
    "brown athletic shoes for women",
    "brown casual shoes for men",
    "looking to buy brown sneakers",
    "find me brown tennis sneakers",
    "searching for brown tennis shoes for ladies",
    "brown sports shoes",
    "brown shoes for guys",
    "brown shoes for ladies",
    "I need tennis shoes in brown color",
    "where are brown men's shoes?",
    "I'd like women's brown shoes",
    "can you show me brown sneakers?",
    "tennis shoes for women in brown",
]

found = 0
for source_query in should_not_match:
    query, score = similar_query(query_embeddings, source_query)
    if query is not None:
        print(source_query, similar_query(query_embeddings, query))
        found += 1
print(found / len(should_not_match))


0.0


### These should match each other

In [ ]:
should_match = [
    "I’m shopping for tennis shoes, maybe in purple",
    "show me purple tennis shoes for women",
    "tennis sneakers that are red",
    "do you have purple tennis shoes?",
    "I'd like to get some purple shoes",
    "shopping for red tennis sneakers",
    "purple women’s gym shoes",
    "red shoes in men's sizes",
    "men’s purple workout sneakers",
    "buy tennis shoes in red",
    "can I find red women’s sneakers?",
    "tennis shoes in purple for guys",
    "do you sell red colored shoes?",
    "get me purple sports shoes",
    "ladies red athletic sneakers",
    "red tennis sneakers for her",
    "tennis shoes for women in red",
    "I'd like to purchase purple shoes",
    "show purple workout shoes for men",
    "tennis shoes in red for adults",
    "purple tennis sneakers for adults",
    "red women's gym sneakers",
    "get women’s tennis shoes, in purple",
    "looking to buy red athletic shoes",
    "tennis sneakers that are purple",
    "purple running shoes for men",
    "I’d love some purple tennis shoes",
    "red sports sneakers for men",
    "purple gym sneakers for women",
    "find red tennis sneakers for guys",
    "looking for red gym shoes",
    "tennis shoes in red for her",
    "please find purple athletic shoes",
    "I'd like tennis shoes for women in purple",
    "buy women's tennis shoes in red"
]

found = 0
for source_query in should_match:
    query, score = similar_query(query_embeddings, source_query)
    if query is not None:
        print(source_query, similar_query(query_embeddings, query))
        found += 1
print(found / len(should_match))

I’m shopping for tennis shoes, maybe in purple ('I’m shopping for tennis shoes, maybe in red', np.float32(1.0000001))
show me purple tennis shoes for women ('tennis shoes for women in purple', np.float32(0.99999994))
tennis sneakers that are red ('red tennis sneakers', np.float32(1.0))
do you have purple tennis shoes? ('can I get some purple tennis shoes?', np.float32(1.0))
I'd like to get some purple shoes ('I want to buy purple shoes', np.float32(1.0000001))
shopping for red tennis sneakers ("shopping for red men's tennis shoes", np.float32(1.0))
purple women’s gym shoes ('purple gym shoes', np.float32(1.0))
red shoes in men's sizes ('red shoes in men’s sizes', np.float32(1.0))
men’s purple workout sneakers ('purple sneakers for men', np.float32(1.0))
buy tennis shoes in red ('tennis shoes that are red', np.float32(0.99999994))
can I find red women’s sneakers? ('can I find red women’s sneakers?', np.float32(0.99999994))
tennis shoes in purple for guys ('tennis shoes, purple, male', n

## With sizes

Numerical attributes can be tricky, here we'll add some shoe sizes to see how well caching performs.

In [ ]:
from sentence_transformers import SentenceTransformer

query_eggshells = [
    "red tennis shoes size 9",
    "tennis shoes, the red ones, size 10",
    "I want to buy red tennis shoes, size 9",
    "please give me red tennis shoes size 10",
    "red shoes for men, size 9",
    "tennis shoes, size 10",
    "red shoes for women, size 9",
    "I want to buy red shoes in size 10",
    "I want to buy red shoes",
    "I want to buy red shoes for men",
    "purple shoes",
    "I want to buy purple shoes",
    "purple tennis shoes",
    "I need purple tennis shoes",
    "tennis shoes in purple",
    "can I get some purple tennis shoes?",
    "I'd like to get some red shoes",
    "red women's tennis shoes",
    "men's red tennis shoes",
    "red shoes men's",
    "shoes, red, for women",
    "give me tennis shoes in red",
    "looking for red shoes for women",
    "shopping for red men's tennis shoes",
    "purple women's shoes",
    "purple men's tennis shoes",
    "I’m shopping for tennis shoes, maybe in red",
    "tennis shoes that are red",
    "show me red tennis shoes for women",
    "do you have red tennis shoes?",
    "red colored shoes",
    "red sneakers",
    "purple sneakers for men",
    "men's purple shoes",
    "women's red sneakers",
    "I'd like women's tennis shoes in red",
    "get me red tennis shoes",
    "can I buy red shoes?",
    "I need tennis shoes for men in red",
    "buy women's tennis shoes purple",
    "want to see red men's shoes",
    "looking for purple tennis shoes for women",
    "show me tennis shoes, red color",
    "are there purple shoes for women?",
    "tennis shoes for women in purple",
    "ladies red tennis shoes",
    "guys' red shoes",
    "purple tennis shoes for ladies",
    "purple shoes for guys",
    "red tennis sneakers",
    "tennis shoes — red ones for women",
    "searching for red sneakers",
    "tennis shoes red male",
    "red tennis shoes female",
    "I’m looking for red sports shoes",
    "purple tennis shoes men's",
    "tennis sneakers in purple for women",
    "buy red sneakers",
    "purple athletic shoes for men",
    "purple gym shoes",
    "red gym shoes",
    "ladies tennis shoes, red color",
    "purple tennis footwear for men",
    "men's gym shoes in red",
    "I'd like to see red shoes for women",
    "I need new red tennis shoes",
    "let’s buy red shoes",
    "do you carry red tennis sneakers?",
    "want red running shoes",
    "get purple tennis sneakers",
    "red tennis shoes for women",
    "red shoes in men’s sizes",
    "purple colored shoes for women",
    "do you have men’s tennis shoes in red?",
    "purple shoes for women please",
    "can I find red women’s sneakers?",
    "I want red tennis shoes for men",
    "men’s shoes in red",
    "show red tennis shoes",
    "women's red running shoes",
    "purple tennis shoes women want",
    "buy men's purple shoes",
    "buy women’s red sneakers",
    "tennis shoes with red color",
    "tennis shoes for ladies in red",
    "give me red athletic shoes",
    "I’m after red tennis sneakers",
    "purple workout shoes",
    "purple tennis shoes I can run in",
    "can I buy red women's shoes?",
    "buy shoes in red for women",
    "looking for tennis shoes, red ones",
    "I like purple tennis shoes",
    "tennis sneakers in red for men",
    "ladies' red shoes",
    "running shoes, red, for women",
    "men’s purple sports shoes",
    "tennis shoes, purple, male",
    "tennis sneakers purple female",
    "purple shoes for sports",
    "purple shoes for tennis",
    "get me red shoes for gym",
    "I’d like tennis shoes, in red"
]

model = SentenceTransformer('intfloat/e5-small-v2')
prefix = "query: "
query_embeddings = model.encode([prefix + query for query in query_eggshells])
print(query_embeddings)

[[-0.06747236  0.04893694  0.02887078 ... -0.02481415 -0.01721771
   0.04565324]
 [-0.07227435  0.06599408  0.04443949 ... -0.00329761 -0.00349636
   0.0638652 ]
 [-0.06643242  0.05211239  0.02513147 ... -0.03110779 -0.0225138
   0.04541651]
 ...
 [-0.10389227  0.03124737  0.01955353 ... -0.01935114  0.00356002
   0.07294314]
 [-0.09446127  0.03613773  0.01525081 ...  0.01329067 -0.07370114
   0.04785777]
 [-0.06866553  0.05815653 -0.00062291 ... -0.00144215 -0.02783558
   0.06176304]]


In [ ]:
should_not_match = [
    "red tennis shoes size 9"
    "show me brown shoes size 9",
    "I'm looking for brown shoes for men size 10",
    "brown shoes for women please size 7",
    "I'd like some brown tennis sneakers",
    "where can I buy brown tennis shoes for women?",
    "get me some tennis shoes, brown ones",
    "tennis sneakers in brown for men",
    "can I find brown sports shoes for women?",
    "looking for brown men's tennis shoes",
    "do you carry women's tennis shoes in brown?",
    "shopping for brown tennis sneakers",
    "brown sneakers for women",
    "brown gym shoes for men",
    "I'd like tennis shoes in brown",
    "please show me brown shoes",
    "tennis shoes, brown color for ladies",
    "can I get brown shoes?",
    "brown colored sneakers",
    "men’s shoes in brown",
    "ladies’ brown tennis sneakers",
    "tennis shoes in brown for women size 12",
    "give me brown sneakers",
    "brown tennis footwear for men",
    "brown athletic shoes for women",
    "brown casual shoes for men",
    "looking to buy brown sneakers",
    "find me brown tennis sneakers",
    "searching for brown tennis shoes for ladies",
    "brown sports shoes",
    "brown shoes for guys",
    "brown shoes for ladies",
    "I need tennis shoes in brown color",
    "where are brown men's shoes?",
    "I'd like women's brown shoes",
    "can you show me brown sneakers?",
    "tennis shoes for women in brown",
]

found = 0
for source_query in should_not_match:
    query, score = similar_query(query_embeddings, source_query)
    if query is not None:
        print(source_query, similar_query(query_embeddings, query))
        found += 1
print(found / len(should_not_match))


red tennis shoes size 9show me brown shoes size 9 ('red tennis shoes size 9', np.float32(1.0000001))
0.027777777777777776


In [ ]:
should_match = [
    "red tennis shoes size 9 red tennis shoes",
    "tennis shoes, the red ones, size 10",
    "I want to buy red tennis shoes, size 9",
    "please give me red tennis shoes size 10",
    "red shoes for men, size 9",
    "tennis shoes, size 10",
    "red shoes for women, size 9",
    "I want to buy red shoes in size 10",
]

found = 0
for source_query in should_match:
    query, score = similar_query(query_embeddings, source_query)
    if query is not None:
        print(source_query, similar_query(query_embeddings, query))
        found += 1
print(found / len(should_not_match))

red tennis shoes size 9 red tennis shoes ('red tennis shoes size 9', np.float32(1.0000001))
tennis shoes, the red ones, size 10 ('tennis shoes, the red ones, size 10', np.float32(1.0000001))
I want to buy red tennis shoes, size 9 ('I want to buy red tennis shoes, size 9', np.float32(1.0))
please give me red tennis shoes size 10 ('please give me red tennis shoes size 10', np.float32(1.0))
red shoes for men, size 9 ('red shoes for men, size 9', np.float32(1.0))
tennis shoes, size 10 ('tennis shoes, size 10', np.float32(1.0))
red shoes for women, size 9 ('red shoes for women, size 9', np.float32(0.99999994))
I want to buy red shoes in size 10 ('I want to buy red shoes in size 10', np.float32(0.99999994))
0.2222222222222222
